In [2]:
import torch
import torch.nn as nn
import argparse
from torchvision import models, transforms
import os
import pycocotools.coco as COCO
import torch.utils.data as data

parser = argparse.ArgumentParser()
parser.add_argument('--gpu', type=str, default='0')
parser.add_argument('--epoch', type=int, default='64')
config = parser.parse_args(['--gpu', '0', '--epoch', '64'])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [3]:
class CocoDataset(data.Dataset):
    def __init__(self, root, vocab, transform, annotation_file=None):
        if annotation_file == None:
            throw ("Load annotation file Error")
        self.root = root
        self.coco = COCO(annotation_file)
        self.vocab = vocab
        self.transform = transform
    
    def __getitem__(self, index):
        image_id = self.coco.ann[index]
        

In [ ]:
class CNNencoder(nn.Module):
    def __init__(self, embedding_size):
        super(CNNencoder, self).__init__()
        resnet = models.resnet152(pretrained=True)
        self.resnet = nn.Sequential(*list(resnet.children())[:-1])
        self.linear = nn.Linear(resnet.fc.in_features, embedding_size)
        self.bn = nn.BatchNorm1d(embedding_size, momentum=0.01)
    
    def forward(self, x):
        with torch.no_grad():
            x = self.resnet(x)
        x = x.view(x.shape[0], -1)
        x = self.bn(self.linear(x))
        return x

class RNNdecoder(nn.Module):
    def __init__(self, embedding_size, hidden_size, voc_size, num_layers, max_length=20):
        super(RNNdecoder, self).__init__()
        self.embedding = nn.Embedding(voc_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first = True)
        self.linear = nn.Linear(hidden_size, voc_size)
        self.max_length = max_length
    
    def forward(self, features, captions, lengths):
        embeddings = self.embedding(captions)
        #每一个都与全局feature进行concat
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        #把序列压紧
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(embedding, lengths, True)
        print(packed_embeddings.shape)
        h, _ = self.rnn(packed_embeddings)
        outputs = self.linear(h[0])
        return outputs

    def sample(self, features, states = None):
        inputs = features.unsqueeze(1)
        output_id = []
        for i in range(self.max_length):
            _, h, states = self.rnn(inputs, states)
            output = self.linear(h.squeeze(1))
            prediction = torch.argmax(output, dim=1)
            output_id.append(prediction)
            inputs = self.embedding(prediction).unsqueeze(1)
        return torch.stack(output_id, 1)
    